In [1]:
import seaborn as sns
sns.set()

In [2]:
from static_grader import grader

# The New York Social Graph


[New York Social Diary](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/) provides a
fascinating lens onto New York's socially well-to-do.  The data forms a natural [social graph](https://en.wikipedia.org/wiki/Social_graph) for New York's social elite.  Take a look at this page of a recent [run-of-the-mill holiday party](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/2014/holiday-dinners-and-doers).

Besides the brand-name celebrities, you will notice the photos have carefully annotated captions labeling those that appear in the photos.  We can think of this as implicitly implying a social graph: there is a connection between two individuals if they appear in a picture together.

For this project, we will assemble the social graph from photo captions for parties _dated December 1, 2014 and before_.  Using this graph, we can make guesses at the most popular socialites, the most influential people, and the most tightly coupled pairs.

These pages are hosted on the Internet Archive, which can be quite slow and unreliable. To get around this, we have created an API that provides the captions. This API lives at `https://party-captions.tditrain.com`. The [documentation](https://party-captions.tditrain.com) describes how this API works in detail. At a high level, it's divided into two parts
- An endpoint that provides a list of parties, `/parties`
- An endpoint that provides the captions for a given party, `/captions`

Both take parameters that allow us to select what we're looking for.

To get the social graph that we want, we'll attack the problem in several steps:
1. Get a list of the parties we want
1. Parse the names from each caption for one party
1. Parse the names for the rest of the parties
1. Assemble the graph

## Getting the parties


The `/parties` endpoint provides us with a list of party names and dates (the date the party occurred). It can only provide up to 100 at a time, and there are over 1000 parties in the data set. By using the `limit` and `offset` parameters, as described in the documentation, get a list of all of the parties and their dates.

As we did in class, we recommend using [`requests`](http://docs.python-requests.org/en/master/) to hit the endpoint. The checkpoints are expecting a list where each element corresponds to one party. How you want to represent this party (as a tuple, a dictionary, or something else) is up to you.

The API will return a JSON object containing a list of party names and dates, and some metadata.  Here is an example, only returning the first ten parties for our convenience.

In [3]:
import requests

requests.get("https://party-captions.tditrain.com/parties?limit=10").json()

{'message': '',
 'next': 'parties?offset=10&limit=10',
 'parties': [{'date': '2015-03-13',
   'name': '2015-bunny-hop-the-boys-club-old-bags-and-more'},
  {'date': '2015-03-11',
   'name': '2015-sab-the-jewish-museum-roundabout-theatre-and-faces'},
  {'date': '2015-03-05',
   'name': '2015-adaa-art-show-bronx-museum-the-china-arts-foundation-international-and-the-palm'},
  {'date': '2015-03-04',
   'name': '2015-the-60th-anniversary-of-the-viennese-opera-ball-and-mcnys-directors-council'},
  {'date': '2015-03-02',
   'name': '2015-the-new-york-philharmonic-the-new-york-botanical-garden-longhouse-reserve-and'},
  {'date': '2015-02-26', 'name': '2015-mission-accomplished'},
  {'date': '2015-02-25', 'name': '2015-philanthropic-endeavors'},
  {'date': '2015-02-23', 'name': '2015-dining-with-the-divas'},
  {'date': '2015-02-18', 'name': '2015-fielding-dreams'},
  {'date': '2015-02-11', 'name': '2015-generosity-leadership'}]}

We want the information in the `parties` element. You will need to call the API multiple times to get all the parties.

In [4]:
party_list_all=[]
req_num=0
while True:
    message_map=requests.get(f"https://party-captions.tditrain.com/parties?limit=100&offset={req_num*100}").json()
    if len(message_map['parties'])==0:
        break
    req_num+=1
    party_list_all.extend(message_map['parties'])

Now that we have our list of parties, we'll need to remove those that occurred after December 1st, 2014 (we keep the ones that occurred _on_ or before that date). The API provided us with the dates, as strings. One option would be to use `datetime`'s `strptime` method and the [format codes for dates](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) to parse this into dates for comparison.

In [25]:
party_list =[]
for item in party_list_all:
    if item['date']>'2014-12-01':
        continue
    party_list.append(item)

In [6]:
# If you have successfully gotten all of the parties, there should be 1145 of them
# Double check that you are not skipping or duplicating parties
# if you are, look at how you are incrementing your offset
# Have you filtered by the date?
grader.check(len(party_list) == 1145)

True

To avoid having to get the party list from the API again if we restart the notebook, we should save this list to a file. There are many ways to do it, here's how with `dill`.

In [7]:
import dill

with open('nysd-parties.pkd', 'wb') as f:
    dill.dump(party_list, f)

And to read it back later, we just `load` it.

In [8]:
with open('nysd-parties.pkd', 'rb') as f:
    party_list = dill.load(f)

## Question 1: histogram


Get the number of party pages for each of the 95 months (that is, month-year pair) in the data.  Represent this histogram as a list of 95 tuples, each of the form `("Dec-2014", 1)`.  Note that you can convert `datetime` objects into these sort of strings with `strftime` and the [format codes for dates](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) from before.

The grader is expecting the list of tuples. 

Plot the histogram for yourself.  Do you see any trends?

In [9]:
from collections import defaultdict
monthCount=defaultdict(int)
from datetime import datetime

for party in party_list:
    monthCount[datetime.strptime(party['date'], '%Y-%m-%d').strftime('%b-%Y')]+=1

histogram=[]
for item in monthCount:
    histogram.append((item, monthCount[item]))
    

#histogram = [("Dec-2014", 1)] * 95  # Replace this fake answer with your real results

In [10]:
grader.score('graph__histogram', histogram)

Your score: 1.0000


## Parsing captions


We now have all of the parties.  For each party, we'll need to get the captions, then find who appears in each caption. Let's start with a single party, [the benefit cocktails and dinner](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood) for [Lenox Hill Neighborhood House](http://www.lenoxhill.org/), a neighborhood organization for the East Side. In our API, this corresponds to the party named `2015-celebrating-the-neighborhood`.  Let's get the captions for it.

In our API, the `/captions` endpoint takes a parameter `party`, which we give the name of the party we want. We can then extract the captions from the JSON it returns.

In [11]:
response = requests.get('https://party-captions.tditrain.com/captions?party=2015-celebrating-the-neighborhood')
captions = response.json()['captions']

We'll need to do this for all of our parties later, so we should make it a function we can call. Take the party name as an argument and return the list of captions. 

We want to avoid having to hit the API repeatedly the next time we need to run the notebook.  While you could save the files by hand, as we did before, a checkpoint library like [ediblepickle](https://pypi.python.org/pypi/ediblepickle/1.1.3) can handle this for you.  (Note, though, that you may not want to enable this until you are sure that your function is working.)

You should also keep in mind that HTTP requests fail occasionally, for transient reasons.  You should plan how to detect and react to these failures.   The [retrying module](https://pypi.python.org/pypi/retrying) is one way to deal with this.

In [12]:
def get_captions(party_name):
    return requests.get(f'https://party-captions.tditrain.com/captions?party={party_name}').json()['captions']

If things have gone according to plan, this should get the same captions as before.

In [13]:
# This cell is expecting get_captions to return a list of the captions themselves
# Other routes to a solution might need to adjust this cell a bit
grader.check(get_captions('2015-celebrating-the-neighborhood') == captions)

True

Now that we have some sample captions, let's start parsing names out of those captions.  There are many ways of going about this, and we leave the details up to you.  Some issues to consider:

  1. Some captions are not useful: they contain long narrative texts that explain the event.  Try to find some heuristic rules to separate captions that are a list of names from those that are not.  A few heuristics include:
    - Look for sentences (which have verbs) and as opposed to lists of nouns. For example, [`nltk` does part of speech tagging](http://www.nltk.org/book/ch05.html) but it is a little slow. There may also be heuristics that accomplish the same thing.
    - Similarly, spaCy's [entity recognition](https://spacy.io/docs/usage/entity-recognition) could be useful here, but like `nltk` using `spaCy` will add to processing time.
    - Look for commonly repeated threads (e.g. you might end up picking up the photo credits or people such as "a friend").
    - Long captions are often not lists of people.  The cutoff is subjective, but for grading purposes **we set that cutoff at 250 characters**.
  1. Many of the captions contain extraneous whitespace or other formatting issues you may need to deal with.
  1. You will want to separate the captions based on various forms of punctuation.  Try using `re.split`, which is more sophisticated than `string.split`. **Note**: The reference solution uses regex exclusively for name parsing.
  1. You might find a person named "ra Lebenthal".  There is no one by this name.  Any idea what might cause that?
  1. This site is pretty formal and likes to say things like "Mayor Michael Bloomberg" after his election but "Michael Bloomberg" before his election.  Can you find other ('optional') titles that are being used?  They should probably be filtered out because they ultimately refer to the same person: "Michael Bloomberg."
  1. There is a special case you might find where couples are written as e.g. "John and Mary Smith". You will need to write some extra logic to make sure this properly parses to two names: "John Smith" and "Mary Smith".
  1. When parsing names from captions, it can help to look at your output frequently and address the problems that you see coming up, iterating until you have a list that looks reasonable. This is the approach used in the reference solution. Because we can only asymptotically approach perfect identification and entity matching, we have to stop somewhere.
  1. Your eye is very good at doing this sort of parsing.  You will find it helpful to look at a caption and the names you parse of out it. Do this for a selection of captions to detect potential issues.
  1. You want to keep the names in a caption together - that's how we can tell they're connected to each other! You should get one list of names for each caption.
  
**Questions worth considering:**
  1. Who is Patrick McMullan and should he be included in the results? How would you address this?
  2. What else could you do to improve the quality of the graph's information?

In [41]:
# You will want to make a function that takes in a caption and returns a list of names
import re

def checkName(name):
    # Too many small letter words in a name, von, van, de, etc. are okay
    nameparts=name.split()
    i=0
    for part in nameparts:
        if ord(part[0])>=ord('a') and ord(part[0])<=ord('z'):
            i+=1
    if len(nameparts)>=2:
        return (i<=1)
    return (i==0)
        
def getnames(caption):
    res=[]
    if len(caption)<=250:
        nameList=re.split(', and |, | with ', caption)
        for name in nameList:
            name=re.split(' as| at| on| \(| &| in', name)[0]
            name=re.split('Mr. |Mrs. |Mayor |Dr. ', name)[-1]
            if len(name.split())>0 and len(name.split("'"))==1 and len(name.split('"'))==1 and len(name.split('The'))==1:
                if len(name.split(' and '))>1:
                    names=[' '.join(split_name.split()) for split_name in name.split(' and ')]
                    if len(names[0].split(' '))<2 and len(names[1].split(' '))==2:
                        names[0]+=(' '+names[1].split(' ')[1])
                    for split_name in names:
                        if checkName(split_name):
                            res.append(split_name)  
                else:
                    if checkName(name):
                        res.append(' '.join(name.split()))
    return res

## Question 2: sample_names


Once you feel that your algorithm is working well, parse all of the captions we got from the `2015-celebrating-the-neighborhood` party and extract all the names mentioned.  Sort them alphabetically, by first name, and return the first hundred unique names.

In [42]:
captions = get_captions('2015-celebrating-the-neighborhood')
all_names=set()
for caption in captions:
    names=getnames(caption)
    for name in names:
        all_names.add(name)

sample_names=sorted(list(all_names))[:100]

grader.score('graph__sample_names', sample_names)

Your score: 1.0000


Now, test your tools on a few other parties.  You will probably find that other parties have new issues in their captions that trip up your caption parser.  But don't worry if the parser isn't perfect - just try to get the easy cases for now. You may need to come back and refine it more for the later questions, however.

## Parsing all the parties


Once you are satisfied that your parser is working, we want to run it for all of our parties. First, get the captions for all of the parties in our party list. If you haven't implemented some caching of the captions, you probably want to do this first.

In [43]:
# It may take several minutes to fetch all the captions
captions=[]
for party in party_list:
    partyCaptions=get_captions(party['name'])
    captions.extend(partyCaptions)


And parse the names in each caption.

In [44]:
# You should have a list of names for each caption
# Depending on how you set up your parser, this may take quite a while
Names=[]
for caption in captions:
    Names.append(getnames(caption))

You should end up with over 100,000 captions and roughly 110,000 names.

In [45]:
print(sum([len(item) for item in Names]), len(captions))

238423 127127


## Building the graph

For the remaining analysis, we think of the problem in terms of a
[network](http://en.wikipedia.org/wiki/Computer_network) or a
[graph](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29).  Any time a pair of people appear in a caption together, that is considered a link.  What we have described is more appropriately called an (undirected)
[multigraph](http://en.wikipedia.org/wiki/Multigraph) with no self-loops, but this has an obvious analog in terms of an undirected [weighted graph](http://en.wikipedia.org/wiki/Graph_%28mathematics%29#Weighted_graph).

In the remainder of this miniproject, we will analyze the social graph of the New York social elite.  We recommend using python's [`networkx`](https://networkx.github.io/) library to build this social graph.

In [61]:
import itertools  # itertools.combinations may be useful
import networkx as nx

weight_map=defaultdict(int)
popularity_map=defaultdict(int)

for nameList in Names:
    for i in range(len(nameList)):
        for j in range(i+1, len(nameList)):
            if nameList[j]<nameList[i]:
                weight_map[(nameList[j], nameList[i])]+=1
            else:
                weight_map[(nameList[i], nameList[j])]+=1
                
print('Total unique connections:', len(weight_map))
for u, v in weight_map:
    popularity_map[u]+=weight_map[(u, v)]
    popularity_map[v]+=weight_map[(u, v)]

G=nx.Graph()
G.add_weighted_edges_from([(u, v, weight_map[(u, v)]) for u, v in weight_map])
    
        

Total unique connections: 195070


You should find you have roughly 200,000 distinct pairs of people appearing in photos together - corresponding to how many (weighted) edges there are in our graph.

## Question 3: degree


The simplest question to ask is "who is the most popular"?  The easiest way to answer this question is to look at how many connections everyone has.  Return the top 100 people and their degree.  Remember that if an edge of the graph has weight 2, it counts for 2 in the degree.

**Checkpoint:** Some aggregate stats on the solution:
    
    count:  100.0
    mean:   202.2
    std:     94.1
    min:    132.0
    25%:    146.8
    50%:    169.5
    75%:    212.0
    max:    713.0
    
Note that these checkpoints are guidelines, you may not match them exactly.

In [62]:
import heapq  # Heaps are efficient structures for tracking the largest
              # elements in a collection.  Use introspection to find the
              # function you need.

minHeap=[]
for item in popularity_map:
    if len(minHeap)<100:
        heapq.heappush(minHeap, [popularity_map[item], item])
    elif minHeap[0][0]<popularity_map[item]:
        heapq.heappushpop(minHeap, [popularity_map[item], item])
        
degree = [(name, count) for count, name in minHeap]

grader.score('graph__degree', degree)

Your score: 0.9300


## Question 4: PageRank


A similar way to determine popularity is to look at their
[PageRank](http://en.wikipedia.org/wiki/PageRank).  PageRank is used for web ranking and was originally
[patented](http://patft.uspto.gov/netacgi/nph-Parser?patentnumber=6285999) by Google and is essentially the stationary distribution of a [Markov
chain](http://en.wikipedia.org/wiki/Markov_chain) implied by the social graph. You can implement this yourself or use the version in `networkx`.

Use 0.85 as the damping parameter so that there is a 15% chance of jumping to another vertex at random.

**Checkpoint:** Some aggregate stats on the solution:


    count:  100.000000
    mean:     0.000193
    std:      0.000080
    min:      0.000130
    25%:      0.000144
    50%:      0.000169
    75%:      0.000206
    max:      0.000646


In [68]:
pagerank = [('Martha Stewart', 0.00019312108706213307)] * 100

pr = nx.pagerank(G, alpha=0.85)

minHeap=[]
for item in pr:
    if len(minHeap)<100:
        heapq.heappush(minHeap, [pr[item], item])
    elif minHeap[0][0]<pr[item]:
        heapq.heappushpop(minHeap, [pr[item], item])
        
pagerank=[(name, rank) for rank, name in minHeap]

grader.score('graph__pagerank', pagerank)

Your score: 0.9100


## Question 5: best_friends


Another interesting question is who tend to co-occur with each other.  Give us the 100 edges with the highest weights.

Google these people and see what their connection is.  Can we use this to detect instances of infidelity?

**Checkpoint:** Some aggregate stats on the solution:

    count:  100.0
    mean:    27.5
    std:     16.9
    min:     14.0
    25%:     17.0
    50%:     21.5
    75%:     31.3
    max:    122.0

In [70]:
best_friends = [(('Michael Kennedy', 'Eleanora Kennedy'), 41)] * 100

minHeap=[]
for u, v in weight_map:
    if len(minHeap)<100:
        heapq.heappush(minHeap, [weight_map[(u, v)], u, v])
    elif minHeap[0][0]<weight_map[(u, v)]:
        heapq.heappushpop(minHeap, [weight_map[(u, v)], u, v])
        
best_friends=[((friend1, friend2), weight) for weight, friend1, friend2 in minHeap]

grader.score('graph__best_friends', best_friends)

Your score: 0.9000


*Copyright &copy; 2022 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*